

# pyJedAI with pyTorch pre-trained embeddings and FAISS

<div align="center"> 
    <img src="https://cdn.icon-icons.com/icons2/2699/PNG/512/pytorch_logo_icon_169823.png" style="width: 250px;"/>
</div>


## How to install?

pyJedAI is an open-source library that can be installed from PyPI.


In [ ]:
!pip install pyjedai -U

In [ ]:
!pip show pyjedai

Imports

In [1]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph

from pyjedai.evaluation import Evaluation
from pyjedai.datamodel import Data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
d1 = pd.read_csv("../data/ccer/D2/abt.csv", sep='|', engine='python', na_filter=False).astype(str)
d2 = pd.read_csv("../data/ccer/D2/buy.csv", sep='|', engine='python', na_filter=False).astype(str)
gt = pd.read_csv("../data/ccer/D2/gt.csv", sep='|', engine='python')

attr1 = d1.columns[1:].to_list()
attr2 = d2.columns[1:].to_list()

data = Data(dataset_1=d1,
            attributes_1=attr1,
            id_column_name_1='id',
            dataset_2=d2,
            attributes_2=attr2,
            id_column_name_2='id',
            ground_truth=gt)

# Block Building

## Pre-trained pyTorch & GENSIM embeddings

Available embeddings:

- Gensim: `{ 'fasttext', 'glove', 'word2vec'}`
- pyTorch Sentence transformers : `{'smpnet','st5','sdistilroberta','sminilm','sent_glove'}`
- pyTorch Word transformers :`{'bert', 'distilbert', 'roberta', 'xlnet', 'albert'}`

## FAISS

faiss.IndexIVFFlat is an implementation of an inverted file index with coarse quantization. This index is used to efficiently search for nearest neighbors of a query vector in a large dataset of vectors. Here's a brief explanation of the parameters used in this index:


In [4]:
from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

In [5]:
emb = EmbeddingsNNBlockBuilding(vectorizer='sminilm',
                                similarity_search='faiss')

blocks, g = emb.build_blocks(data,
                             top_k=5,
                             similarity_distance='euclidean',
                             load_embeddings_if_exist=False,
                             save_embeddings=False,
                             with_entity_matching=True)

Building blocks via Embeddings-NN Block Building [sminilm, faiss]


Embeddings-NN Block Building [sminilm, faiss]:   0%|          | 0/2152 [00:00<?, ?it/s]

Device selected:  cpu


In [6]:
emb.evaluate(blocks, with_classification_report=True, with_stats=True)

***************************************************************************************************************************
                                         Μethod:  Embeddings-NN Block Building
***************************************************************************************************************************
Method name: Embeddings-NN Block Building
Parameters: 
	Vectorizer: sminilm
	Similarity-Search: faiss
	Top-K: 5
	Vector size: 384
Runtime: 160.6854 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:      9.38% 
	Recall:        93.77%
	F1-score:      17.05%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 1009
	False positives: 9751
	True negatives: 1156633
	False negatives: 67
	Total comparisons: 10760
──────────────────────────────────────────────

{'Precision %': 9.37732342007435,
 'Recall %': 93.77323420074349,
 'F1 %': 17.049678945589726,
 'True Positives': 1009,
 'False Positives': 9751,
 'True Negatives': 1156633,
 'False Negatives': 67}

# Entity Clustering

In [7]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering

In [8]:
ccc = UniqueMappingClustering()
clusters = ccc.process(g, data, similarity_threshold=0.63)

In [9]:
ccc.evaluate(clusters, with_classification_report=True)

***************************************************************************************************************************
                                         Μethod:  Unique Mapping Clustering
***************************************************************************************************************************
Method name: Unique Mapping Clustering
Parameters: 
Runtime: 0.1010 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:     83.18% 
	Recall:        67.10%
	F1-score:      74.28%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 722
	False positives: 146
	True negatives: 1156346
	False negatives: 354
	Total comparisons: 868
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


{'Precision %': 83.17972350230414,
 'Recall %': 67.1003717472119,
 'F1 %': 74.2798353909465,
 'True Positives': 722,
 'False Positives': 146,
 'True Negatives': 1156346,
 'False Negatives': 354}

<hr>
<div align="right">
K. Nikoletos, G. Papadakis & M. Koubarakis
</div>
<div align="right">
<a href="https://github.com/Nikoletos-K/pyJedAI/blob/main/LICENSE">Apache License 2.0</a>
</div>